# Multiple Linear Regression

## Objectives

- Dealing with categorical variables

- Diagnosis of the model fit

In [1]:
import statsmodels.api as sm
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

In [2]:
# load Credit data (from https://www.kaggle.com/avikpaul4u/credit-card-balance)
df = pd.read_csv('data/Credit.csv', index_col=0)
df = df.drop(columns='Ethnicity')
df.head()

,Income,Limit,Rating,Cards,Age,Education,Gender,Student,Married,Balance
1,14.891,3606,283,2,34,11,Male,No,Yes,333
2,106.025,6645,483,3,82,15,Female,Yes,Yes,903
3,104.593,7075,514,4,71,11,Male,No,No,580
4,148.924,9504,681,3,36,11,Female,No,No,964
5,55.882,4897,357,2,68,16,Male,No,Yes,331


In [ ]:
# Define the problem
outcome = 'Balance'
x_cols = ['Income', 'Limit', 'Rating', 'Cards', 'Age', 'Education', 'Gender',
          'Student', 'Married']

In [ ]:
# Problem: let's calculate multiple regression witout categorical variables

## Dealing with Categorical Variables


### With Pandas `get_dummies()`

In [ ]:
categorical_variables = ['Gender', 'Student', 'Married']

In [ ]:
# one hot encode variables
df_ohe = pd.get_dummies(df[x_cols],
                        columns=categorical_variables,
                        drop_first=True
)
print(df_ohe.shape)
df_ohe.head(3)

### With `sklearn` One Hot Encoder

In [ ]:
from sklearn.preprocessing import OneHotEncoder

from sklearn.compose import ColumnTransformer

In [ ]:
# create an encoder object. This will help us to convert
# categorical variables to new columns
encoder = OneHotEncoder(handle_unknown='error',
                        drop='first',
                        categories='auto')

# Create an columntransformer object.
# This will help us to merge transformed columns
# with the rest of the dataset.

ct = ColumnTransformer(transformers=[('ohe', encoder, categorical_variables)],
                       remainder='passthrough')
ct.fit(df[x_cols])
X = ct.transform(df[x_cols])

In [ ]:
X.shape

Now we let's try to understand what it means to add categorical variables to our model.

In [ ]:
X

In [ ]:
X = df_ohe
X.head(3)

In [ ]:
y = df.Balance.values

Xconst = sm.add_constant(X)
model = sm.OLS(y, Xconst, hasconst=True)
result = model.fit()
result.summary()

Note that the $R^{2}$ and $R^{2}_{adj}$ increased significantly but at the same time some predictors p_values are not significant anymore.

W can easily convert it to an adjusted $R^{2}$ by using the formula:

$$ \bar{R}^{2} = 1 - (1- R^{2})\frac{n-1}{n-p-1}$$

where $p$ is the total number of features used to train model.

[Wikipedia-$R^{2}$](https://en.wikipedia.org/wiki/Coefficient_of_determination#Adjusted_R2)

### Multicollinearity

As we discussed before the multicollinearity is a problem for interprettability and confidence intervels. Now we will see how to detect multicolinearity and how to solve this problem.

[Statistics by Jim - Multicollinearity in linear regression](https://statisticsbyjim.com/regression/multicollinearity-in-regression-analysis/)


[Wikipedia VIF - Calculation and Analysis](https://en.wikipedia.org/wiki/Variance_inflation_factor)

In [ ]:
# One way of detecting multicollinearity is Variance inflation factor.

from statsmodels.stats.outliers_influence import variance_inflation_factor

In [ ]:
X.head(3)

In [ ]:
vif = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]
list(zip(df.columns, vif))

In [ ]:
# Let's try to show very quickly that our interpretation is correct

In the literature, you might see that it is suggested that if VIF> 10 then this column can  be dropped. Some other resources say if VIF > 5 it is ok to drop a column. Note that these corresponds to $R^{2} = 0.90$ and $R^{2} = 0.80$ respectively. In this case, we can consider to drop 'weight' feature as its VIF >5. Let's see how this effects the model.

In [ ]:
X_without_rating = X.drop(columns=['Rating'])

In [ ]:
X_without_rating.head()

In [ ]:
Xconst = sm.add_constant(X_without_rating)

model = sm.OLS(y, Xconst, hasconst= True)

res = model.fit()

res.summary()

In [ ]:
vif = [variance_inflation_factor(X_without_rating.values, i) for i in range(X_without_rating.shape[1])]
list(zip(X_without_rating.columns.tolist(), vif))

## Feature Selection

In [ ]:
## sklearn.feature_selection has a class
## called RFE for recursive feature selection

from sklearn.feature_selection import RFE
from sklearn.linear_model import LinearRegression

For more details and the documentation of RFE you can check:

[sklearn- Recursive Feature Selection](https://scikit-learn.org/stable/modules/generated/sklearn.feature_selection.RFE.html)

[Feature Selection in sklearn -RFE](https://scikit-learn.org/stable/modules/feature_selection.html#rfe)

[Other methods in sklearn fo](https://scikit-learn.org/stable/modules/feature_selection.html#)

In [ ]:
## instantiate the linear regression object
lm = LinearRegression()
## instantiate the selector object
selector = RFE(lm, n_features_to_select=8)

## fit the model
selector.fit(X,y)

## check which columns are selected
ind = selector.get_support()
print(ind)

## Note that we got different columns
X.loc[:, ind].head()

## Further Reading

To address the collinearity in multiple linear regression we can also use methods like Principal Component Analysis (PCA) and Partial Least Squares (PLS).

[Wikipedia- Partial Least Squares ](https://en.wikipedia.org/wiki/Partial_least_squares_regression)

[Partial Least Squares](http://www.statsoft.com/Textbook/Partial-Least-Squares)

[sklearn - Partial Least Squares](https://scikit-learn.org/stable/modules/generated/sklearn.cross_decomposition.PLSRegression.html#sklearn.cross_decomposition.PLSRegression)